# Imports

In [ ]:
import numpy as np
import pandas as pd
import datetime
from sklearn.preprocessing import MinMaxScaler
from tensorflow import keras
from tensorflow.keras.layers import Dense, LSTM, Dropout,Flatten
from tensorflow.keras import Sequential
#from keras.preprocessing.sequence import TimeseriesGenerator
import matplotlib.pyplot as plt

In [ ]:
ds = pd.read_csv("/kaggle/input/bitcoin-historical-data/bitstampUSD_1-min_data_2012-01-01_to_2021-03-31.csv")

In [ ]:
ds["Date"] = pd.to_datetime(ds["Timestamp"],unit="s").dt.date
data=ds.groupby("Date").mean()
data.head()

In [ ]:
price_by_date=data["Weighted_Price"].rename("Price")
price_by_date.plot()

# Timesteps Preparation

In [ ]:
######## Prepare timesteps groups ########
time_window = 15 # Time series are of (time_window - time_ahead) size
time_ahead = 1 # Short forecast
scaler=MinMaxScaler()
length = len(price_by_date)
price_scaled=scaler.fit_transform(pd.DataFrame(price_by_date))[:,-1]
price_groups = []
for i in range(0,length-time_window):
    price_groups.append(price_scaled[i:i+time_window])

price_groups = np.array(price_groups)
price_groups=price_groups[...,np.newaxis]

### Splitting the groups into X and Y goes something like this:
![](https://i.ibb.co/54CgsMH/XY.jpg)

In [ ]:
######## Split Xs, and Ys########
split_rate = 0.95
length = len(price_groups)
X_train = price_groups[:int(length*(split_rate)), :-time_ahead]
X_test =  price_groups[int(length*(split_rate)):, :-time_ahead]
y_train = price_groups[:int(length*(split_rate)),  -time_ahead:]
y_test =  price_groups[int(length*(split_rate)):,  -time_ahead:]
print(X_train.shape,y_train.shape)
print(X_test.shape,y_test.shape)

# Compile and Fit Model

In [ ]:
######## Model ########
model = keras.models.Sequential([
     keras.layers.LSTM(128,activation='relu', return_sequences=True, input_shape=(None,1)),
     keras.layers.LSTM(128,activation='relu'),
     keras.layers.Dense(time_ahead)
])

model.compile(loss='mse', optimizer="adam", metrics=['mse'])
history = model.fit(X_train, y_train, epochs=40, verbose=1,  validation_data=(X_test, y_test))

# Predictions

In [ ]:
y_pred = model.predict(X_test)[:, 0][..., np.newaxis]
y_last = price_scaled.flatten()[-time_window:]
y_pred30future=[]
for _ in range(30):
    newX = np.reshape(y_last[-time_window:],(1,time_window,1))
    y_pred30future.append(model.predict(newX)[0,0])
    y_last=np.append(y_last,np.array(y_pred30future)[-1])


In [ ]:
######## Make scaled numbers normal again ########
y_pred_inv = scaler.inverse_transform(y_pred)
y_pred30future_inv = scaler.inverse_transform(np.reshape(y_pred30future,(len(y_pred30future),1))) 
y_test_inv = scaler.inverse_transform(y_test[:,-1])

######## Plot Real, Prediction, and 30 Days after ########
plt.plot(range(len(y_test)),y_test_inv,"r-", label="Real")
plt.plot(range(len(y_pred_inv)), y_pred_inv, "b-",label="Prediction")
plt.plot(range(len(y_pred_inv),len(y_pred_inv)+len(y_pred30future_inv)), y_pred30future_inv, "g-",label="30 Days Future")
plt.legend(fontsize=12)
plt.show()

Seems my model doesn't like Bitcoin's future a lot. Then again, as far as unpredictable intangible assets go, Bitcoin is definitively up there.